In [8]:
## File for generating 3 folds for readmission and nextvisittask
from sklearn.model_selection import KFold

In [3]:
n_folds = 3

In [4]:
import sys
sys.path.insert(1, '../')
from utils.packages import *

/var/local/home/linler17/.conda/envs/masterenv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def fix_length(data, visit_label):
    df = data.copy()
    df['hadm_id'] = df['hadm_id'].apply(lambda x: x[:visit_label])
    df['medication_code'] = df['medication_code'].apply(lambda x: x[:visit_label])
    df['diagnos_code'] = df['diagnos_code'].apply(lambda x: x[:visit_label])
    df['procedure_code'] = df['procedure_code'].apply(lambda x: x[:visit_label])
    
    return df

## Readmission

#### MIMIC

In [56]:
labelvisit = 10

In [57]:
data_path = '../data/datasets/MIMIC/mimic_done3.parquet'
save_path = '../data/cross_val/MIMIC/readmission_visit{}'.format(labelvisit) 

In [58]:
df = pd.read_parquet(data_path).rename(columns={'ccsr':'diagnos_code', 'ndc':'medication_code', 'procedure_ccsr':'procedure_code'})
df['age'] =df['age'].apply(lambda x: list(map(int, x)))
df = df[df['hadm_id'].map(len) >= labelvisit]
df = fix_length(df, labelvisit)

In [59]:
kfold = KFold(n_splits=3, shuffle=True)

for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(df)):
    train, test = df.iloc[train_idx, :], df.iloc[test_idx, :]
    train.to_parquet(save_path + '/fold{}/train.parquet'.format(fold_idx + 1))
    test.to_parquet(save_path + '/fold{}/test.parquet'.format(fold_idx + 1))

#### Synthea

In [46]:
labelvisit = 10

In [47]:
data_path = '../data/datasets/Synthea/Final_cohorts/synthea_done.parquet'
save_path = '../data/cross_val/Synthea/readmission_visit{}'.format(labelvisit) 

In [48]:
df_synth = pd.read_parquet(data_path)
df_synth = df_synth[df_synth['hadm_id'].map(len) >= labelvisit]
df_synth = fix_length(df_synth, labelvisit)

In [89]:
kfold = KFold(n_splits=3, shuffle=True)

for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(df_synth)):
    print(train_idx)
    train, test = df_synth.iloc[train_idx, :], df_synth.iloc[test_idx, :]
    train.to_parquet(save_path + '/fold{}/train.parquet'.format(fold_idx + 1))
    test.to_parquet(save_path + '/fold{}/test.parquet'.format(fold_idx + 1))

[     2      4      5 ... 348174 348175 348176]
[     0      1      3 ... 348173 348175 348176]
[     0      1      2 ... 348172 348173 348174]


In [87]:
train1 = pd.read_parquet(save_path + '/fold{}/train.parquet'.format(1))
train2 = pd.read_parquet(save_path + '/fold{}/train.parquet'.format(2))

## NextVisit

#### MIMIC

In [73]:
nextvisitlabel = 5

In [74]:
data_path = '../data/datasets/MIMIC/mimic_done3.parquet'
save_path = '../data/cross_val/MIMIC/nextvisit{}'.format(nextvisitlabel) 

In [75]:
df = pd.read_parquet(data_path).rename(columns={'ccsr':'diagnos_code', 'ndc':'medication_code', 'procedure_ccsr':'procedure_code'})
df['age'] =df['age'].apply(lambda x: list(map(int, x)))
df = df[df['hadm_id'].map(len) >= nextvisitlabel]
df = fix_length(df, nextvisitlabel)

In [76]:
kfold = KFold(n_splits=3, shuffle=True)

for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(df)):
    train, test = df.iloc[train_idx, :], df.iloc[test_idx, :]
    train.to_parquet(save_path + '/fold{}/train.parquet'.format(fold_idx + 1))
    test.to_parquet(save_path + '/fold{}/test.parquet'.format(fold_idx + 1))
    
    totdata = pd.concat([train, test])
    label_codes = totdata['diagnos_code'].apply(lambda x: x[-1]).tolist()
    label_codes = np.concatenate(label_codes, axis=0)
    
    np.save(save_path + '/fold{}/Nextvisit_{}_labelcodes_fold{}.npy'.format(fold_idx + 1, nextvisitlabel, fold_idx), label_codes)

In [ ]:
label_codes = data['diagnos_code'].apply(lambda x: x[-1]).tolist()
label_codes = np.concatenate(label_codes, axis=0)

#### Synthea

In [77]:
nextvisitlabel = 5

In [78]:
data_path = '../data/datasets/Synthea/Final_cohorts/synthea_done.parquet'
save_path = '../data/cross_val/Synthea/nextvisit{}'.format(nextvisitlabel) 

In [79]:
df_synth = pd.read_parquet(data_path)
df_synth = df_synth[df_synth['hadm_id'].map(len) >= nextvisitlabel]
df_synth = fix_length(df_synth, nextvisitlabel)

In [80]:
kfold = KFold(n_splits=3, shuffle=True)

for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(df_synth)):
    train, test = df_synth.iloc[train_idx, :], df_synth.iloc[test_idx, :]
    train.to_parquet(save_path + '/fold{}/train.parquet'.format(fold_idx + 1))
    test.to_parquet(save_path + '/fold{}/test.parquet'.format(fold_idx + 1))
    
    totdata = pd.concat([train, test])
    label_codes = totdata['diagnos_code'].apply(lambda x: x[-1]).tolist()
    label_codes = np.concatenate(label_codes, axis=0)
    
    np.save(save_path + '/fold{}/Nextvisit_{}_labelcodes_fold{}.npy'.format(fold_idx + 1, nextvisitlabel, fold_idx), label_codes)